# Probabilities through monte carlo simulations

Here, rather than training a model to predict the $E[\text{dice roll score}]$, I just build a monte carlo simulation. 

In [1]:
import sys
sys.path.insert(0, '../../src')

In [2]:
import json
import random
from time import perf_counter
from collections import defaultdict

from farkle.logic import gameobjects as go

In [3]:
dh = go.DiceHand(2,3)
dh.possible_scores()

[]

In [14]:
# for saving dicehands as json
class DiceHandEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, go.DiceHand):
            return obj.json_encode
        return super().default(obj)
    
def as_dicehand(dct):
    if '__DiceHand__' in dct:
        return go.DiceHand.json_decode(dct)
    return dct

In [15]:
j = json.dumps({'dh': dh}, cls=DiceHandEncoder)

In [16]:
j

'{"dh": {"__DiceHand__": true, "free": [2, 3], "locked": [], "score": 0}}'

In [17]:
jl = json.loads(j, object_hook=as_dicehand)

In [18]:
jl

{'dh': DiceHand(free=[2, 3], locked=[], score=0)}

In [19]:
type(jl['dh'])

farkle.logic.gameobjects.DiceHand

In [11]:
d = {0: 'a', 1: 'b'}

In [13]:
str(d)

"{0: 'a', 1: 'b'}"

In [14]:
print(d)

{0: 'a', 1: 'b'}


In [4]:
class MonteCarloSimulation(object):
    def __init__(self, max_dice: int = 6):
        self.max_dice = max_dice
        self.dice_range = range(1, self.max_dice + 1)
        self.dice_hands = {i: go.DiceHand(num_dice=i) for i in self.dice_range}
        
        self.roll_obs = {i: 0 for i in self.dice_range}
        self.possible_score_obs = {i: 0 for i in self.dice_range}
        self.possible_score_frequencies = {i: defaultdict(int) for i in self.dice_range}
        
    def roll(self, num_dice: int):
        self.dice_hands[num_dice].roll()
        self.roll_obs[num_dice] += 1
        if self.dice_hands[num_dice].possible_scores():
            for dh in self.dice_hands[num_dice].possible_scores():
                dh: go.DiceHand
                self.possible_score_obs[num_dice] += 1
                self.possible_score_frequencies[num_dice][dh] += 1
        else:
            self.possible_score_frequencies[num_dice][None] += 1
            
    def roll_all(self, num_rolls: int = 1):
        """Rolls all hands in ratio where 1 dice is 1 roll and 
        each additional die is 6x more rolls"""
        for die_num in self.dice_range:
            for roll_num in range(6**(die_num-1)):
                self.roll(die_num)
    
    def P_farkle(self, num_dice: int):
        return self.possible_score_frequencies[num_dice][None] / self.roll_obs[num_dice]
    
    def E_score(self, num_dice: int):
        E_score = 0
        for dh, freq in self.possible_score_frequencies[num_dice].items():
            if dh: E_score += dh.score * freq
        return (E_score / self.possible_score_obs[num_dice]) * (1 - self.P_farkle(num_dice))
    
    def json_encode(self):
        d = {'max_dice': self.max_dice, 
             'roll_obs': self.roll_obs, 
             'possible_score_obs': self.possible_score_obs}

In [5]:
mcs = MonteCarloSimulation()

In [6]:
mcs.dice_hands[3]

DiceHand(free=[3, 3, 5], locked=[], score=0)

In [7]:
''.split(', ')

['']

In [13]:
'asdf'.find('sd')

1

In [14]:
'asdf'.find('f')

3

In [15]:
'asdf'[1:3]

'sd'

In [17]:
[1,2,3] + [4,5]

[1, 2, 3, 4, 5]

In [18]:
for i in range(1, 5):
    print(i)

1
2
3
4


In [19]:
def asdf(*args):
    for i in args:
        print(i)

In [20]:
asdf(1,2,4)

1
2
4


In [21]:
asdf(*[1,3,6])

1
3
6


In [22]:
mcs.dice_hands[6]

DiceHand(6, 6, 2, 6, 5, 6, score=0)

In [25]:
t0 = perf_counter()
for i in range(6**5):
    if i % 300 == 0:
        print(i)
    mcs.roll(6)
t1 = perf_counter()
print(round(t1-t0, 4))

0
300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900
7200
7500
476.1664


In [27]:
mcs.possible_score_frequencies[6]

defaultdict(int,
            {DiceHand(1, score=100): 5317,
             DiceHand(1, 1, score=200): 2139,
             DiceHand(1, 1, 5, score=250): 1176,
             DiceHand(1, 5, score=150): 3316,
             DiceHand(5, score=50): 5308,
             DiceHand(5, 5, score=100): 2111,
             DiceHand(5, 5, 5, score=150): 510,
             DiceHand(5, 5, 5, score=500): 510,
             DiceHand(1, 5, 5, score=200): 1166,
             DiceHand(1, 1, 2, 2, 2, 5, score=450): 10,
             DiceHand(1, 1, 2, 2, 2, score=400): 46,
             DiceHand(1, 2, 2, 2, 5, score=350): 88,
             DiceHand(1, 2, 2, 2, score=300): 239,
             DiceHand(5, 2, 2, 2, score=250): 233,
             DiceHand(2, 2, 2, score=200): 480,
             DiceHand(1, 1, 1, score=300): 549,
             DiceHand(1, 1, 1, 5, score=350): 240,
             DiceHand(5, 1, 1, 1, score=1050): 240,
             DiceHand(1, 1, 1, score=1000): 549,
             DiceHand(1, 6, 6, 6, score=700): 230,
   

In [13]:
mcs.roll(3)

In [14]:
mcs.possible_score_frequencies[3]

defaultdict(int, {DiceHand(1, score=100): 1})

In [15]:
for i in range(4):
    mcs.roll(3)

In [16]:
mcs.possible_score_frequencies[3]

defaultdict(int,
            {DiceHand(1, score=100): 1, DiceHand(5, score=50): 3, None: 1})

In [4]:
class MonteCarloDiceHand(go.DiceHand):
    def __init__(self, num_dice):
        super().__init__(num_dice=num_dice)

        self.possible_score_frequencies = defaultdict(int)
        self.roll_obs = 0
        self.possible_score_obs = 0

    def roll(self):
        super().roll()
        self.roll_obs += 1
        # save resulting possible scores if any otherwise None
        if self.possible_scores():
            for dh in self.possible_scores():
                dh: go.DiceHand
                self.possible_score_obs += 1
                self.possible_score_frequencies[dh] += 1
        else:
            self.possible_score_frequencies[None] += 1

    @property
    def E_any_score(self):
        return 1 - (self.possible_score_frequencies[None] / self.roll_obs)

    @property
    def E_score_arithmetic(self):
        E_score = 0
        for dh, freq in self.possible_score_frequencies.items():
            if dh: E_score += dh.score * freq
        return (E_score / self.possible_score_obs) * self.E_any_score

In [5]:
mcdh = MonteCarloDiceHand(3)

In [6]:
mcdh

DiceHand(4, 4, 4, score=0)

In [7]:
mcdh.roll()

In [8]:
mcdh

DiceHand(1, 2, 2, score=0)

In [9]:
mcdh.possible_score_frequencies

defaultdict(int, {DiceHand(1, score=100): 1})

In [10]:
for i in range(4):
    mcdh.roll()

In [11]:
mcdh.possible_score_frequencies

defaultdict(int,
            {DiceHand(1, score=100): 1,
             None: 3,
             DiceHand(5, score=50): 1,
             DiceHand(5, 5, score=100): 1})

In [13]:
mcdh.roll_obs

5

In [14]:
mcdh.E_any_score

0.4

In [15]:
mcdh.E_score_arithmetic

33.333333333333336